<a href="https://colab.research.google.com/github/BlackAngel2108/Pytorch_train/blob/main/Iris_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris
import torch

iris = load_iris()
_global_var_data_x = torch.tensor(iris.data, dtype=torch.float32)
_global_var_target = torch.tensor(iris.target, dtype=torch.float32)

In [2]:
_global_var_data_x.size()[0]

150

In [3]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class IrisDataset(data.Dataset):
    def __init__(self):
        self.data = _global_var_data_x # тензор размерностью (150, 4), тип float32
        self.target = _global_var_target # тензор размерностью (150, ), тип int64 (long)

        self.length = _global_var_data_x.size()[0]
        self.categories = ['setosa' 'versicolor' 'virginica'] # названия классов
        self.features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']

    def __getitem__(self, item):
        x = self.data[item]
        y = self.target[item]
        return x,y

    def __len__(self):
        return self.length


class IrisClassModel(nn.Module):
    def __init__(self, in_features=4, out_features=3):
        super().__init__()
        self.layer1 = nn.Linear(in_features, 16)
        self.layer2 = nn.Linear(16, out_features)

    def forward(self, x):
        x = self.layer1(x).relu()
        x = self.layer2(x)
        return x

torch.manual_seed(11)
model = IrisClassModel(4,3)
model.train()

epochs = 10 # число эпох обучения
batch_size = 8 # размер батча

# создать объект класса IrisDataset
d_train = IrisDataset()
train_data = data.DataLoader(d_train,batch_size, shuffle = True ) # создать объект класса DataLoader с размером пакетов batch_size и перемешиванием образов выборки

optimizer = optim.Adam(model.parameters(), lr = 0.01) # создать оптимизатор Adam для обучения модели с шагом обучения 0.01
loss_func = nn.CrossEntropyLoss() # создать функцию потерь с помощью класса CrossEntropyLoss (используется при многоклассовой классификации)

for _e in range(epochs): # итерации по эпохам
    loss_mean = 0
    lm_count = 0

    train_tqdm = tqdm(train_data, leave = True)
    for x_train, y_train in train_tqdm:
        predict = model(x_train) # вычислить прогноз модели для данных x_train
        loss = loss_func(predict, y_train.long() ) # вычислить значение функции потерь

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count+=1
        loss_mean = loss_mean + (loss.item() - loss_mean) / lm_count
        train_tqdm.set_description(f"Epoch [{_e+1}/{epochs}], loss_mean = {loss_mean:.3f}")

model.eval()
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for x, y in d_train:
        output = model(x.unsqueeze(0))
        _, predicted = torch.max(output, 1)
        total_predictions += 1
        if predicted == y:
            correct_predictions += 1

Q = correct_predictions / total_predictions

Epoch [10/10], loss_mean = 0.164: 100%|██████████| 19/19 [00:00<00:00, 228.74it/s]


In [4]:
Q

0.96